# Sztuczne sieci neuronowe - laboratorium 3

#### Dane studenta: Natalia Chostenko

In [1]:
import torch

### Pytania kontrolne

1. Jakie znasz funkcje straty (w regresji i w klasyfikacji)?
2. Jakie znasz algorytmy (min. jeden, wraz z wariantami) minimalizacji funkcji straty?
3. Czym jest gradient funkcji?
4. Wymień znane Ci hiperparametry stosowane w algorytmie najszybszego spadku.
5. Jakie problemy mogą wystąpić przy nieodpowiednim doborze hiperparametrów tego algorytmu?


### Przypomnienie poprzedniego ćwiczenia

Poprzednie ćwiczenie:
> Załóżmy, że mamy dwa termometry:
> - jeden mierzy temperaturę w stopniach Celsjusza
> - drugi mierzy temperaturę w nieznanej nam skali, ale jest bardzo ładny i chcemy go powiesić na ścianie
>
> Zanim to zrobimy, chcemy się dowiedzieć, jak przeliczać wskazania drugiego termometru na stopnie Celsjusza.
> Spróbujemy znaleźć "wzór" tego przekształcenia na podstawie pomiarów dokonanych obydwoma termometrami.

W poprzednim ćwiczeniu:
- na podstawie wizualizacji danych wybraliśmy do tego problemu model liniowy
- zaimplementowaliśmy go jako funkcję `model` (dwa parametry: `w` i `b`)
- zdefiniowaliśmy funkcję straty dla regresji liniowej - błąd średniokwadratowy (funkcja `loss_fn`)
- na potrzeby obliczenia gradientu funkcji straty (wektora pochodnych względem parametrów modelu, `dL/dw`) zdefiniowailśmy funkcje obliczające poszczególne komponenty reguły łańcuchowej (`dL/dw = (dL/dtp) * (dtp/dw)`)
- podjęliśmy próbę minimalizacji funkcji straty algorytmem najszybszego spadku
- eksperymentowaliśmy z wartością stałej uczącej (`learning_rate`)
- zauważyliśmy konieczność normalizacji danych
- na podstawie wyznaczonych wartości parametrów znaleźliśmy przekształcenie ze skali Fahrenheita do skali Celsjusza

Dzisiaj dowiemy się, jak niektóre z tych kroków zrealizować "automatycznie" z użyciem PyTorch `autograd` i `torch.optim`.

In [2]:
data_unknown = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4]
data_celsius = [0.5,  14.0, 15.0, 28.0, 11.0,  8.0,  3.0, -4.0,  6.0, 13.0, 21.0]

t_u = torch.tensor(data_unknown)
t_c = torch.tensor(data_celsius)


In [3]:
def model(t_u, w, b):
    return w * t_u + b

In [4]:
def loss_fn(t_p, t_c):
    squared_diffs = (t_p - t_c) ** 2
    return squared_diffs.mean()

### Normalizacja danych

W poprzednim ćwiczeniu, aby algorytm najszybszego spadku nie "wybuchł", należało znormalizować dane wejściowe. Zgodnie z oryginalnym przykładem z książki "Deep Learning with PyTorch" normalizacja polegała na dziesięciokrotnym pomniejszeniu wejść (aby sprowadzić dane do "bezpieczniejszego" przedziału). Ta niestandardowa normalizacja wprowadziła jednak nieco niepotrzebnego chaosu do zajęć.

Dlatego dzisiaj zrobimy inaczej - poddamy dane wejściowe standaryzacji.


#### Ćwiczenie
Dokonaj standaryzacji danych wejściowych i przypisz je do tensora `t_un`.

In [5]:
# mean i std
t_u_mean = t_u.mean()
t_u_std = t_u.std()

# standaryzacja
t_un = (t_u - t_u_mean) / t_u_std

print(t_un)

tensor([-0.9565,  0.2436,  0.3802,  1.7883,  0.2673, -0.1723, -1.0635, -1.7823,
        -0.2020,  0.5109,  0.9862])


### PyTorch autograd

W przypadku regresji liniowej dla jednej zmiennej wejściowej i dwóch parametrów, "ręczne" obliczenie pochodnych nie było szczególnie uciążliwe. Dla bardziej skomplikowanych modeli (a takimi będą sieci neuronowe) warto użyć narzędzia, które może wykonać tę pracę za nas.

`autograd` to wbudowany w PyTorch silnik do obliczania pochodnych funkcji - także złożonych.

Poza przechowywaniem wartości liczbowych, tensory w PyTorch mogą zapamiętywać, poprzez jakie operacje i z których (innych) tensorów powstały, tworząc strukturę grafu obliczeń ("computation graph"). Na podstawie tego grafu `autograd` może następnie obliczyć pochodne względem poszczególnych parametrów modelu.

Wartości pochodnych będą zapisane w atrybucie `.grad` tensora. Aby włączyć działanie `autograd` dla konkretnego tensora (oraz wszystkich tensorów, które z niego powstają w wyniku różnych operacji), należy przy tworzeniu go podać argument `requires_grad=True`.

#### Ćwiczenie
Stwórz tensor `params` zawierający początkowe wartości parametrów: w = 1 i b = 0. Zadbaj o włączenie `autograd` dla tego tensora. Sprawdź wartość `params.grad` zaraz po utworzeniu tensora.

In [6]:
params = torch.tensor([1.0, 0.0], requires_grad=True)

print(params.grad)

None


### Aktualizacja gradientów

Zawartość atrybutu `grad` aktualizuje się po wywołaniu funkcji `loss.backward()` (gdzie `loss` to tensor reprezentujący  funkcję straty po przejściu "w przód" przez graf).

Wyjaśnienie:  
Podczas obliczania funkcji straty (gdy dla `params` ustawimy `requires_grad=True`), poza samymi obliczeniami tworzy się graf reprezentujący poszczególne operacje jako wierzchołki (końcowym wierzchołkiem jest `loss`). Po wywołaniu `loss.backward()` graf przetwarzany jest wstecz i obliczane są pochodne względem poszczególnych parametrów.

#### Ćwiczenie
Użyj przygotowanych funkcji `model` i `loss_fn`, aby stworzyć graf obliczeń i uzyskać tensor reprezentujący funkcję straty, a następnie uruchom na nim metodę `backward()`. Sprawdź wartość `params.grad`. Sprawdź też (np. z użyciem `print`), jaka jest zawartość tensorów funkcji straty i predykcji.

In [7]:
#predykcja
t_p = model(t_un, *params)

#funkcja straty
loss = loss_fn(t_p, t_c)
loss.backward()

print("gradienty params", params.grad)
print("funkcja straty", loss)
print("predykcja", t_p)




gradienty params tensor([-14.6089, -21.0000])
funkcja straty tensor(171.8683, grad_fn=<MeanBackward0>)
predykcja tensor([-0.9565,  0.2436,  0.3802,  1.7883,  0.2673, -0.1723, -1.0635, -1.7823,
        -0.2020,  0.5109,  0.9862], grad_fn=<AddBackward0>)


### Akumulacja gradientów
Uwaga:  
Ze względów praktycznych (np. ze względu na zastosowanie w przypadku ograniczonej pamięci), każde wywołanie `loss.backward()` powoduje akumulację  gradientów (dodanie nowo obliczonych do już istniejących wartości w `.grad`, a nie nadpisanie). Zwykle pożądanym zachowaniem jest jednak użycie jedynie gradientów z aktualnej iteracji. W tym celu należy "wyzerować" gradienty, np. na początku każdej iteracji algorytmu optymalizacji.

#### Ćwiczenie
Do wyzerowania gradientów na początku każdej iteracji można użyć funkcji `.zero()`.
Wyzeruj gradienty tensora `params`. Pomyśl, jak zabezpieczyć się przed błędem, który może się pojawić w pierwszej iteracji algorytmu optymalizacji.

Uwaga:  
W PyTorchu obowiązuje konwencja, że funkcje z `_` na końcu nazwy służą do wywoływania operacji "w miejscu" (in-place) - nie tworzą kopii tensora / widoku na pamieć, ale zmieniają jej zawartość. W ćwiczeniu użyj wariantu "in-place" funkcji zerującej tensor.


w pierwszej iteracji gradienty mogą jeszcze nie istniec, co spowoduje bląd, w zwiazku z tym przed wywolaniem .zero_ mozna sprawdzic czy params.grad nie jest None.

In [48]:
if params.grad is not None:
    params.grad.zero_()
print(params.grad)

tensor([0., 0.])


#### Ćwiczenie
Korzystając z kodu funkcji `training_loop` z poprzedniego ćwiczenia (przeklejonej poniżej), napisz funkcję `training_loop_autograd`, w której ręczne obliczanie gradientu zastąpi użycie silnika `autograd`.

Uwaga:  
Krok aktualizacji tensora parametrów należy dodatkowo zamknąć w bloku `with torch.no_grad()`: (aby "na chwilę" wyłączyć autograd na potrzeby zmiany zawartości tensora `params`).

In [9]:
def training_loop(n_iters, learning_rate, params, t_u, t_c):
    for iteration in range(1, n_iters+1):
        w, b = params
        t_p = model(t_u, *params)
        loss = loss_fn(t_p, t_c)
        grad = grad_fn(t_u, t_c, t_p, w, b)
        
        # tzw. "vanilla" gradient descent        
        params = params - learning_rate * grad
        
        print('After iteration %d, Loss %f' % (iteration, float(loss)))
    
    return params

In [49]:
def training_loop_autograd(n_iters, learning_rate, params, t_u, t_c):
    for iteration in range(1, n_iters + 1):
        # nie rozpakowujemy manualnie paraemtrow
        t_p = model(t_u, *params)
        loss = loss_fn(t_p, t_c)

        # zerowanie gradientow
        if params.grad is not None:
            params.grad.zero_()
        
        # liczenie gradientow
        loss.backward()

        # Aktualizacja parametrow bez sledzenia przez autograd
        with torch.no_grad():
            params -= learning_rate * params.grad

        if iteration % 500 == 0:  # zeby nie wypisywac wszystkiego
            print('After iteration %d, Loss %f' % (iteration, float(loss)))
    
    return params


#### Ćwiczenie
Uruchom `training_loop_autograd` dla 1000 iteracji, ustaw `learning_rate = 5e-3` i jako dane wejściowe podaj znormalizowany tensor `t_un`. Zainicjalizuj parametry tensorem [1,0, 0.0] (włącz dla niego autograd).

In [50]:
learning_rate = 5e-3
n_iters = 1000


params = torch.tensor([1.0, 0.0], requires_grad=True)
trained_params = training_loop_autograd(n_iters, learning_rate, params, t_un, t_c)

trained_params


After iteration 500, Loss 2.938963
After iteration 1000, Loss 2.927647


tensor([ 9.0340, 10.4995], requires_grad=True)

#### Ćwiczenie
Zapisz równanie transformujące oryginalne dane wejściowe (`t_u`) do stopni Celsjusza (`t_p`).
Znajdź współczynniki prostej opisującej model liniowy. Porównaj z wynikiem z poprzednich zajęć.

Uwaga / hint:  
Uwzględnij średnią i odchylenie standardowe policzone dla danych wejściowych na początku dzisiejszych zajęć.

trzeba przeksztalcic rownanie liniowe do tej postaci:

t_p = t_u * w + b


In [14]:
w, b = params
w = w / t_u.std()
b = b - w * t_u.mean()
print(w.item(),b.item())

0.5367202758789062 -17.30256462097168


### torch.optim

Algorytm najszybszego spadku jest przykładem jednego z wielu algorytmów optymalizacji dostępnych w PyTorch. Implementacje różnych optymalizatorów znajdują się w module `torch.optim`.

Przykłady optymalizatorów w PyTorch:
- `SGD` - Stochastic Gradient Descent (tak naprawdę tzw. Batch Gradient Descent), z dodatkową opcją tzw. "momentum"
- `Adagrad`, `RMSProp` - "adaptive, per-parameter learning rate"
- `Adam` - Adaptive Moment Estimation - połączenie SGD+momentum z RMSProp - obecnie bardzo popularny
- ...

Bardziej szczegółowo omówimy te algorytmy w późniejszym czasie.

In [15]:
import torch.optim as optim
dir(optim)

['ASGD',
 'Adadelta',
 'Adagrad',
 'Adam',
 'AdamW',
 'Adamax',
 'LBFGS',
 'NAdam',
 'Optimizer',
 'RAdam',
 'RMSprop',
 'Rprop',
 'SGD',
 'SparseAdam',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_functional',
 '_multi_tensor',
 'lr_scheduler',
 'swa_utils']

### Optymalizacja funkcji straty w torch.optim

Optymalizator inicjalizuje się wybierając konkretną klasę z dostępnych w `torch.optim`. Pierwszym argumentem przy tworzeniu optymalizatora jest lista parametrów modelu (nie zawsze wszystkich - czasem np. "zamraża" się niektóre parametry i nie aktualizuje ich). Tworząc obiekt optymalizatora podaje się też jako argument m.in. wartość learning rate (argument `lr`).

W API optymalizatora są dwie metody: `zero_grad()` oraz `step()`. Pierwsza z nich zeruje gradienty parametrów (patrz wyżej), a druga wykonuje krok aktualizacji parametrów zgodny z wybranym algorytmem.

#### Ćwiczenie
Zainicjalizuj tensor `params` jako [1.0, 0.0] (z włączonym autograd) oraz optymalizator SGD ze stałą uczącą o wartości 0.01.


In [22]:
params = torch.tensor([1.0, 0.0], requires_grad=True)
optimizer = optim.SGD([params], lr=0.01)

#### Ćwiczenie
Na podstawie `training_loop_autograd` (napisanej wyżej) napisz funkcję `training_loop_optim`, w której zerowanie gradientów i krok aktualizacji parametrów wykonane będą z użyciem optymalizatora z `torch.optim`.

In [51]:
def training_loop_optim(n_iters, optimizer, params, t_u, t_c):
    for iteration in range(1, n_iters + 1):
        # nie rozpakowujemy manualnie paraemtrow
        t_p = model(t_u, *params)
        loss = loss_fn(t_p, t_c)

        # zerowanie gradientow
        optimizer.zero_grad()
        
        # liczenie gradientow
        loss.backward()

        # krok aktualizacji parametrow z optymalizatorem
        optimizer.step()

        if iteration % 500 == 0:  # zeby nie wypisywac wszystkiego 
            print('After iteration %d, Loss %f' % (iteration, float(loss)))
    
    return params


#### Ćwiczenie
Uruchom `training_loop_optim` i porównaj wynik z `training_loop_autograd`. Powinno wyjść to samo.

In [30]:
params = training_loop_optim(n_iters, optimizer, params, t_un, t_c)
print(params)

After iteration 500, Loss: 2.9276463985443115
After iteration 1000, Loss: 2.9276463985443115
tensor([ 9.0340, 10.4995], requires_grad=True)


#### Ćwiczenie
Sprawdź, jak w porównaniu do `SGD` poradzi sobie optymalizator `Adam` dla `learning_rate = 0.01` dla nieznormalizowanego (oryginalnego) tensora danych wejściowych `t_u`.

In [42]:
params = torch.tensor([1.0, 0.0], requires_grad=True)
optimizer = optim.Adam([params], lr=0.01)
params = training_loop_optim(n_iters, optimizer, params, t_u, t_c)
print(params)

After iteration 500, Loss: 25.590320587158203
After iteration 1000, Loss: 22.958574295043945
tensor([ 0.2700, -2.1840], requires_grad=True)


### Podsumowanie:
- zamiast obliczać gradienty "ręcznie", możemy użyć silnika `autograd`
- tensory korzystające z `autograd` (`requires_grad=True`) posiadają atrybut `grad`
- wykonywanie operacji na tensorach powoduje tworzenie grafu obliczeń (computation graph)
- wywołanie funkcji `loss.backward()` na tensorze reprezentującym funkcję straty powoduje aktualizację gradientów (zmienia się zawartość `.grad` dla tensorów w grafie)
- ze względu na akumulację gradientów typowo należy je wyzerować np. na początku każdej iteracji treningu
- moduł `torch.optim` zawiera implementacje algorytmów optymalizacji funkcji straty, m.in. SGD, RMSProp, Adam
- przy inicjalizacji optymalizatora należy podać jako argument zestaw parametrów modelu do optymalizacji oraz wartość stałej uczącej
- korzystając z `torch.optim` w pętli treningowej należy wywołać `optim.zero_grad()` (zerowanie gradientów) oraz `optim.step()` (krok aktualizacji parametrów)

Podczas poprzednich i dzisiejszych ćwiczeń poznaliśmy tak naprawdę podstawowy wariant mechanizmu uczenia sieci neuronowych. Sieci neuronowe to po prostu bardziej złożone modele niż model liniowy, posiadają one dużo więcej parametrów i są modelami nieliniowymi. W sieciach neuronowych w PyTorch także tworzy się graf obliczeń na tensorach, który służy następnie do obliczania gradientów funkcji straty z użyciem `autograd`. Krok aktualizacji parametrów wykonuje się typowo z użyciem jednego z dostępnych w `torch.optim` algorytmów.